In [1]:
import os
import shutil
import numpy as np
import pandas as pd
import ast

In [2]:
exp_dir = "F://Cambridge//MENG//UROP2022//Data//SyMBac//T100-400_iter60_lysis0//"
mask_dir = exp_dir + "masks\\"
channel_image_dir = exp_dir + "convolutions\\"
lineage_dir = exp_dir + "truth\\"
save_dir = exp_dir + "down_sample\\"
if not os.path.isdir(save_dir):
    os.mkdir(save_dir)

### Down sample

In [3]:
mask_files = [f for f in os.listdir(mask_dir) 
              if os.path.isfile(os.path.join(mask_dir, f))]
if not os.path.isdir(save_dir + "masks\\"):
    os.mkdir(save_dir + "masks\\")
for i in range(len(mask_files) // 2):
    shutil.copy(mask_dir + mask_files[i * 2], save_dir + "masks\\synth_{:05d}.tif".format(i))
    
intensity_files = [f for f in os.listdir(channel_image_dir)
                   if os.path.isfile(os.path.join(channel_image_dir, f))]
if not os.path.isdir(save_dir + "convolutions\\"):
    os.mkdir(save_dir + "convolutions\\")
for i in range(len(intensity_files) // 2):
    shutil.copy(channel_image_dir + intensity_files[i * 2], save_dir + "convolutions\\synth_{:05d}.tif".format(i))

# cannot just down sample the true lineage like this
csv_files = [f for f in os.listdir(lineage_dir)
                   if os.path.isfile(os.path.join(lineage_dir, f))]
df = pd.read_csv(lineage_dir + csv_files[0], converters={
            "label": ast.literal_eval,
            "parent_label": ast.literal_eval,
            "centroid": ast.literal_eval,
            "barcode": ast.literal_eval,
            "poles": ast.literal_eval
        })
parent_label = []
odd_time = False
for index, row in df.iterrows():
    if row["time_(mins)"] % 8 == 4: # if down sample the third time
        if odd_time:
            parent_label = []
        parent_label.append(row["parent_label"])
        df.drop(index=index, inplace=True)
        odd_time = False
    elif parent_label:
#         print(row)
#         print(row["time_(mins)"])
#         print(row["parent_label"])
#         print(parent_label)
        df.at[index, "parent_label"] = parent_label[int(row["parent_label"]) - 1]
        odd_time = True
if not os.path.isdir(save_dir + "truth\\"):
    os.mkdir(save_dir + "truth\\")
df.to_csv(save_dir + "truth\\" + csv_files[0])

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'F://Cambridge//MENG//UROP2022//Data//SyMBac//T100-400_iter60_lysis0//truth\\'